# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [3]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?
填補mean值結果最好

In [4]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.69602991289767624

In [5]:
# 空值補 0, 做羅吉斯迴歸
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.69715350840329415

In [6]:
# 空值補 mean, 做羅吉斯迴歸
df_mean = df.fillna(df.mean())
train_X = df_mean[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.69817610337234692

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?
填補0後，用MinMaxScaler，再做Logistic Regression，效果最好

In [7]:
fill_in = ['0','-1','mean']
scaling = ['ori','minmax','std']
result_set = {}
for fill in fill_in:
    if fill == '0':
        df_temp = df.fillna(0)
    elif fill == '-1':
        df_temp = df.fillna(-1)
    if fill == 'mean':
        df_temp = df.fillna(df.mean())
    result = []
    for scale_method in scaling:
        if scale_method == 'ori':
            df_temp = df_temp
        elif scale_method == 'minmax':
            df_temp = MinMaxScaler().fit_transform(df_temp)
        elif scale_method == 'std':
            df_temp = StandardScaler().fit_transform(df_temp)
        train_X = df_temp[:train_num]
        estimator = LogisticRegression()
        pred = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
        result.append(pred)
    result_set[fill] = result

result_df = pd.DataFrame(result_set,index=scaling)
result_df

,-1,0,mean
ori,0.696030,0.697154,0.698176
minmax,0.697135,0.701629,0.699350
std,0.698258,0.698258,0.695941


In [8]:
# 空值補 -1, StandardScaler, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
df_m1 = StandardScaler().fit_transform(df_m1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.69825820177197784

# My Pracetice

In [9]:
df.mean()

Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64

In [10]:
for col in df:
    print(f'mean of {col} is : {df[col].dropna().mean()}')

mean of Pclass is : 2.294881588999236
mean of Age is : 29.881137667304014
mean of SibSp is : 0.4988540870893812
mean of Parch is : 0.3850267379679144
mean of Fare is : 33.2954792813456
